In [1]:
import requests
import pandas as pd
from datetime import datetime

In [2]:

# Toronto Open Data is stored in a CKAN instance. It's APIs are documented here:
# https://docs.ckan.org/en/latest/api/

# To hit our API, you'll be making requests to:
base_url = "https://ckan0.cf.opendata.inter.prod-toronto.ca"

# Datasets are called "packages". Each package can contain many "resources"
# To retrieve the metadata for this package and its resources, use the package name in this page's URL:
url = base_url + "/api/3/action/package_show"
params = { "id": "toronto-island-ferry-ticket-counts"}
package = requests.get(url, params = params).json()


In [3]:
f'Package success: {package["success"]}'

'Package success: True'

In [4]:
print(f'Number of resources: {package["result"]["num_resources"]}\n')
for idx, resource in enumerate(package["result"]["resources"]):
    print(f'{idx}: {resource["name"]}\n**Active: {resource["datastore_active"]}\n')

Number of resources: 4

0: Toronto Island Ferry Ticket Counts
**Active: True

1: Toronto Island Ferry Ticket Counts.csv
**Active: False

2: Toronto Island Ferry Ticket Counts.xml
**Active: False

3: Toronto Island Ferry Ticket Counts.json
**Active: False



In [5]:

# To get resource data:
for idx, resource in enumerate(package["result"]["resources"]):

    # To get data from cached csv:
    if not resource["datastore_active"] and resource['format']=='CSV':
        print(f'**getting data from cached csv**')
        print(f'\t{resource["url"]}')
        df = pd.read_csv(resource["url"])

df.head()

**getting data from cached csv**
	https://ckan0.cf.opendata.inter.prod-toronto.ca/dataset/toronto-island-ferry-ticket-counts/resource/c46719f5-8006-44e1-8b1e-5ad90bb9f6f4/download/toronto-island-ferry-ticket-counts.csv


,_id,Timestamp,Redemption Count,Sales Count
0,1,2024-10-23T10:45:00,22,25
1,2,2024-10-23T10:30:00,36,26
2,3,2024-10-23T10:15:00,69,63
3,4,2024-10-23T10:00:00,4,9
4,5,2024-10-23T09:45:00,8,8


In [6]:
# Parse Timestamp as datetime obj
df['datetimeTimestamp'] = pd.to_datetime(df['Timestamp'], format="%Y-%m-%dT%H:%M:%S")
# Get latest date
latest = max(df['datetimeTimestamp']).date()
print(f"latest:\t{latest}")
# Get today's date
today = datetime.today().date()
print(f"today:\t{today}")

# Checking if the date part of timestamp is the same as today and latest date, add as df columns
df['isToday'] = today == df['datetimeTimestamp'].dt.date
df['isLatest'] = latest == df['datetimeTimestamp'].dt.date
df

latest:	2024-10-23
today:	2024-10-23


,_id,Timestamp,Redemption Count,Sales Count,datetimeTimestamp,isToday,isLatest
0,1,2024-10-23T10:45:00,22,25,2024-10-23 10:45:00,True,True
1,2,2024-10-23T10:30:00,36,26,2024-10-23 10:30:00,True,True
2,3,2024-10-23T10:15:00,69,63,2024-10-23 10:15:00,True,True
3,4,2024-10-23T10:00:00,4,9,2024-10-23 10:00:00,True,True
4,5,2024-10-23T09:45:00,8,8,2024-10-23 09:45:00,True,True
...,...,...,...,...,...,...,...
231392,231393,2015-05-04T16:00:00,0,2,2015-05-04 16:00:00,False,False
231393,231394,2015-05-01T16:00:00,1,0,2015-05-01 16:00:00,False,False
231394,231395,2015-05-01T15:45:00,0,1,2015-05-01 15:45:00,False,False
231395,231396,2015-05-01T15:15:00,0,2,2015-05-01 15:15:00,False,False


In [7]:
# Add time-based features
df['month'] = df['datetimeTimestamp'].dt.month
df['year'] = df['datetimeTimestamp'].dt.year
df['day_of_week'] = df['datetimeTimestamp'].dt.day_name()
df['hour'] = df['datetimeTimestamp'].dt.hour


In [8]:
# Add time-based features
df['month'] = df['datetimeTimestamp'].dt.month
df['year'] = df['datetimeTimestamp'].dt.year
df['season'] = pd.cut(
    df['datetimeTimestamp'].dt.month, 
    bins=[0,3,6,9,12], 
    labels=['Winter', 'Spring', 'Summer', 'Fall']
)
df['hour'] = df['datetimeTimestamp'].dt.hour
df['day_of_week'] = df['datetimeTimestamp'].dt.day_name()

# Create monthly aggregations
monthly_stats = df.groupby(['year', 'month'], observed=True).agg({
    'Sales Count': 'sum',
    'Redemption Count': 'sum'
}).reset_index()

# Calculate monthly averages by season
seasonal_patterns = df.groupby(['season', 'month'], observed=True).agg({
    'Sales Count': 'mean',
    'Redemption Count': 'mean'
}).round(2)

In [9]:
# Peak usage analysis by season
seasonal_hourly = df.groupby(['season', 'hour'], observed=True).agg({
    'Sales Count': 'mean',
    'Redemption Count': 'mean'
}).round(2)

# Find peak hours for each season
peak_hours = seasonal_hourly.groupby('season', observed=True)['Redemption Count'].idxmax()

In [10]:
# Calculate year-over-year growth
yearly_stats = df.groupby('year')[['Sales Count', 'Redemption Count']].sum()
yearly_growth = yearly_stats.pct_change() * 100

In [11]:
# Calculate YoY growth for each month
monthly_stats['previous_year_sales'] = monthly_stats.groupby('month')['Sales Count'].shift(1)
monthly_stats['yoy_growth'] = ((monthly_stats['Sales Count'] - monthly_stats['previous_year_sales']) 
                              / monthly_stats['previous_year_sales'] * 100).round(2)

In [12]:
yearly_stats

,Sales Count,Redemption Count
year,,
2015,1189620,978707
2016,1518428,1425779
2017,682346,698861
2018,1463589,1493560
2019,1249725,1278505
2020,366606,374546
2021,782368,773040
2022,1346659,1349778
2023,1491473,1502883


In [13]:
yearly_growth

,Sales Count,Redemption Count
year,,
2015,NaN,NaN
2016,27.639751,45.679861
2017,-55.062341,-50.983918
2018,114.493673,113.713457
2019,-14.612299,-14.398819
2020,-70.665066,-70.704377
2021,113.408400,106.393874
2022,72.126033,74.606489
2023,10.753576,11.342976


In [14]:
# Update saved csv
df.to_csv(
    'Toronto Island Ferry Ticket Counts.csv',
    header=True,
    index=False
)